In [1]:
import os
import gnupg

In [2]:
CLIENTS = 4
CLIENT_GRAPH = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: [1, 2],
}

In [3]:
def initialize_gpg_agents(CLIENTS):
    client_deets = {}
    for cl in range(CLIENTS):
        folder = "/home/hitesh/.gnupg"+str(cl)
        
        #Make directory
        cmd1 = "mkdir "+folder
        os.system(cmd1)
        
        #Generate key
        gpg = gnupg.GPG(gnupghome=folder)
        input_data = gpg.gen_key_input(
            name_email='client'+str(cl)+'@xyz.com',
            passphrase='1234')
        key = gpg.gen_key(input_data)
        
        client_deets[cl] = {
                            'folder': folder, 
                            'email': 'client'+str(cl)+'@xyz.com', 
                            'passphrase':'1234',
                            'key': key
                           }
    return client_deets

In [4]:
CLIENT_DEETS = initialize_gpg_agents(CLIENTS)
CLIENT_DEETS

{0: {'folder': '/home/hitesh/.gnupg0',
  'email': 'client0@xyz.com',
  'passphrase': '1234',
  'key': <gnupg.GenKey at 0x7f85a02894f0>},
 1: {'folder': '/home/hitesh/.gnupg1',
  'email': 'client1@xyz.com',
  'passphrase': '1234',
  'key': <gnupg.GenKey at 0x7f85a0282e50>},
 2: {'folder': '/home/hitesh/.gnupg2',
  'email': 'client2@xyz.com',
  'passphrase': '1234',
  'key': <gnupg.GenKey at 0x7f85a0269a30>},
 3: {'folder': '/home/hitesh/.gnupg3',
  'email': 'client3@xyz.com',
  'passphrase': '1234',
  'key': <gnupg.GenKey at 0x7f85a0251f10>}}

In [5]:
def share_keys(client_graph, client_deets):
    for client, client_nbs in client_graph.items():
        
        #EXPORT
        gpg = gnupg.GPG(gnupghome=client_deets[client]['folder'])
        ascii_armored_public_keys = gpg.export_keys(client_deets[client]['key'].fingerprint)
        with open('mykeyfile.asc', 'w') as f:
            f.write(ascii_armored_public_keys)
        
        #IMPORT
        for nb in client_nbs:
            gpg = gnupg.GPG(gnupghome=client_deets[nb]['folder'])
            key_data = open('mykeyfile.asc').read()
            import_result = gpg.import_keys(key_data)

In [6]:
share_keys(CLIENT_GRAPH, client_deets)

In [9]:
# gpg = gnupg.GPG(gnupghome=client_deets[2]['folder'])
# gpg.list_keys()

In [25]:
def encrypt_file(client_graph, client_deets, model, node):
    gpg = gnupg.GPG(gnupghome=client_deets[node]['folder'])
    with open(model, 'rb') as f:
        nb_emails = [client_deets[nb]['email'] for nb in client_graph[node]]
        status = gpg.encrypt_file(f, recipients=nb_emails, output = model+'.gpg', always_trust=True)
    return status.status

In [26]:
encrypt_file(CLIENT_GRAPH, CLIENT_DEETS, 'model0.json', 0)

'encryption ok'

In [23]:
def decrypt_file(client_graph, client_deets, model, node):
    gpg = gnupg.GPG(gnupghome=client_deets[node]['folder'])
    with open(model, 'rb') as f:
        status = gpg.decrypt_file(f, passphrase='1234', output=model+'.json')
    return status.status

In [27]:
decrypt_file(CLIENT_GRAPH, CLIENT_DEETS, 'model0.json.gpg', 1)

'decryption ok'